In [1]:

# independent variables - 
# start from 1 min averages
# stop wherever it hangs

# concat other features
# concat averages
# concat 1 min rsis
# concat 1 min macd


# dependent features
# category

# write everything to s3 location
# train.py
# sagemaker notebook results

In [2]:
# convert this into pipeline

In [3]:
# experiment with settings
# correlation analysis

In [4]:
import yaml
import boto3
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [5]:
import os
os.getcwd()

'/home/ec2-user/SageMaker/sisyphus/AthenaInsights/notebooks/data_cleaning'

In [6]:
def load_yaml(loc):
    with open(loc) as stream:
        try:
            print(f'reading from {loc}')
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

### independent variables

In [7]:
client = boto3.client('s3')

In [8]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [9]:
all_paths_config_loc = '../../config/spy_30min_v1/features.yaml'
all_paths_config = load_yaml(all_paths_config_loc)
all_paths_config.keys()

reading from ../../config/spy_30min_v1/features.yaml


dict_keys(['dependent_variable', 'feature_store'])

In [10]:
all_paths_config['feature_store']['1min'].keys()

dict_keys(['stock_bars_1min_base', 'stock_bars_1min_base_macd', 'stock_bars_1min_base_other_features', 'stock_bars_1min_base_rsi', 'stock_bars_1min_diff', 'stock_bars_1min_diff_macd', 'stock_bars_1min_diff_rsi'])

In [11]:
all_paths_config['feature_store']['100D'].keys()

dict_keys(['stock_bars_100D_base', 'stock_bars_100D_base_macd', 'stock_bars_100D_base_rsi', 'stock_bars_100D_diff', 'stock_bars_100D_diff_macd', 'stock_bars_100D_diff_rsi'])

#### loading one min data

In [12]:
cols = eval(all_paths_config['feature_store']['1min']['stock_bars_1min_base']['cols'])
path = all_paths_config['feature_store']['1min']['stock_bars_1min_base']['path']
path, len(cols)

('s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base.parquet',
 87)

In [13]:
cols

['open',
 'high',
 'low',
 'close',
 'volume',
 'symbol',
 'time_delta',
 'open_ema_2m',
 'open_sma_2m',
 'high_ema_2m',
 'high_sma_2m',
 'low_ema_2m',
 'low_sma_2m',
 'close_ema_2m',
 'close_sma_2m',
 'volume_ema_2m',
 'volume_sma_2m',
 'open_ema_3m',
 'open_sma_3m',
 'high_ema_3m',
 'high_sma_3m',
 'low_ema_3m',
 'low_sma_3m',
 'close_ema_3m',
 'close_sma_3m',
 'volume_ema_3m',
 'volume_sma_3m',
 'open_ema_5m',
 'open_sma_5m',
 'high_ema_5m',
 'high_sma_5m',
 'low_ema_5m',
 'low_sma_5m',
 'close_ema_5m',
 'close_sma_5m',
 'volume_ema_5m',
 'volume_sma_5m',
 'open_ema_8m',
 'open_sma_8m',
 'high_ema_8m',
 'high_sma_8m',
 'low_ema_8m',
 'low_sma_8m',
 'close_ema_8m',
 'close_sma_8m',
 'volume_ema_8m',
 'volume_sma_8m',
 'open_ema_12m',
 'open_sma_12m',
 'high_ema_12m',
 'high_sma_12m',
 'low_ema_12m',
 'low_sma_12m',
 'close_ema_12m',
 'close_sma_12m',
 'volume_ema_12m',
 'volume_sma_12m',
 'open_ema_10m',
 'open_sma_10m',
 'high_ema_10m',
 'high_sma_10m',
 'low_ema_10m',
 'low_sma_10m

In [14]:
import pyarrow.dataset as ds
import pyarrow.parquet as pq
one_min_df = pq.read_pandas(path,).to_pandas() # pq.read_pandas('example.parquet', columns=['two']).to_pandas()
one_min_df = one_min_df[one_min_df.symbol=='SPY']
one_min_df.head()
# df = ds.dataset(path).scanner().head(1).to_pandas()

,open,high,low,close,volume,symbol,time_delta,open_ema_2m,open_sma_2m,high_ema_2m,...,open_ema_26m,open_sma_26m,high_ema_26m,high_sma_26m,low_ema_26m,low_sma_26m,close_ema_26m,close_sma_26m,volume_ema_26m,volume_sma_26m
us_eastern_timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 04:00:00,476.25,476.36,476.00,476.31,20460.0,SPY,NaN,476.250000,NaN,476.360000,...,476.250000,NaN,476.360000,NaN,476.000000,NaN,476.310000,NaN,20460.000000,NaN
2024-01-02 04:01:00,476.34,476.34,476.29,476.29,6369.0,SPY,NaN,476.310000,476.295,476.346667,...,476.256667,NaN,476.358519,NaN,476.021481,NaN,476.308519,NaN,19416.222222,NaN
2024-01-02 04:02:00,476.29,476.29,476.28,476.28,6152.0,SPY,NaN,476.296667,476.315,476.308889,...,476.259136,NaN,476.353443,NaN,476.040631,NaN,476.306406,NaN,18433.687243,NaN
2024-01-02 04:03:00,476.27,476.27,476.27,476.27,369.0,SPY,NaN,476.278889,476.280,476.282963,...,476.259941,NaN,476.347262,NaN,476.057621,NaN,476.303709,NaN,17095.562262,NaN
2024-01-02 04:04:00,476.27,476.27,476.27,476.27,369.0,SPY,NaN,476.272963,476.270,476.274321,...,476.260686,NaN,476.341539,NaN,476.073353,NaN,476.301212,NaN,15856.557650,NaN


#### other features

In [15]:
cols = eval(all_paths_config['feature_store']['1min']['stock_bars_1min_base_other_features']['cols'])
path = all_paths_config['feature_store']['1min']['stock_bars_1min_base_other_features']['path']
path, len(cols)

('s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_other_features.parquet',
 181)

In [16]:
cols = [x for x in cols if 'ema' not in x and 'sma' not in x and x not in ['us_eastern_timestamp', 'open', 'high', 'low', 'close', 'volume',]]
len(cols), cols

(95,
 ['open_lag_1',
  'high_lag_1',
  'low_lag_1',
  'close_lag_1',
  'open_lag_2',
  'high_lag_2',
  'low_lag_2',
  'close_lag_2',
  'open_lag_3',
  'high_lag_3',
  'low_lag_3',
  'close_lag_3',
  'open_lag_4',
  'high_lag_4',
  'low_lag_4',
  'close_lag_4',
  'open_lag_5',
  'high_lag_5',
  'low_lag_5',
  'close_lag_5',
  'open_lag_6',
  'high_lag_6',
  'low_lag_6',
  'close_lag_6',
  'open_lag_7',
  'high_lag_7',
  'low_lag_7',
  'close_lag_7',
  'open_lag_8',
  'high_lag_8',
  'low_lag_8',
  'close_lag_8',
  'open_lag_9',
  'high_lag_9',
  'low_lag_9',
  'close_lag_9',
  'open_lag_10',
  'high_lag_10',
  'low_lag_10',
  'close_lag_10',
  'open_lag_11',
  'high_lag_11',
  'low_lag_11',
  'close_lag_11',
  'open_lag_12',
  'high_lag_12',
  'low_lag_12',
  'close_lag_12',
  'open_lag_13',
  'high_lag_13',
  'low_lag_13',
  'close_lag_13',
  'open_lag_14',
  'high_lag_14',
  'low_lag_14',
  'close_lag_14',
  'open_lag_15',
  'high_lag_15',
  'low_lag_15',
  'close_lag_15',
  'price_ch

In [17]:
other_features = pq.read_pandas(path, columns=cols).to_pandas() # pq.read_pandas('example.parquet', columns=['two']).to_pandas()
other_features.head()

,open_lag_1,high_lag_1,low_lag_1,close_lag_1,open_lag_2,high_lag_2,low_lag_2,close_lag_2,open_lag_3,high_lag_3,...,cumulative_local_max_15,cumulative_local_min_15,time_since_prev_max_15,time_since_prev_min_15,local_max_30,local_min_30,cumulative_local_max_30,cumulative_local_min_30,time_since_prev_max_30,time_since_prev_min_30
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,False,False,0,0,0,0
1,476.25,476.36,476.00,476.31,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,1,False,False,0,0,1,1
2,476.34,476.34,476.29,476.29,476.25,476.36,476.00,476.31,NaN,NaN,...,0,0,2,2,False,False,0,0,2,2
3,476.29,476.29,476.28,476.28,476.34,476.34,476.29,476.29,476.25,476.36,...,0,0,3,3,False,False,0,0,3,3
4,476.27,476.27,476.27,476.27,476.29,476.29,476.28,476.28,476.34,476.34,...,0,0,4,4,False,False,0,0,4,4


In [18]:
one_min_df.shape[0]==other_features.shape[0]

True

In [19]:
one_min_df = pd.concat([one_min_df.reset_index(), other_features], axis=1, )

In [20]:
one_min_df.shape, other_features.shape

((214794, 183), (214794, 95))

### dependent_variable

In [21]:
cols = eval(all_paths_config['dependent_variable']['1min']['stock_bars_1min']['cols'])
path = all_paths_config['dependent_variable']['1min']['stock_bars_1min']['path']
path, len(cols), cols

('s3://sisyphus-general-bucket/AthenaInsights/latest_data/dependent_variable/stock_bars_1min.parquet',
 9,
 ['open',
  'high',
  'low',
  'close',
  'close_sma_5m',
  'category',
  'future_highs',
  'future_lows',
  'slopes'])

In [22]:
dependent_var_df = pq.read_pandas(path, columns=['category']).to_pandas() # pq.read_pandas('example.parquet', columns=['two']).to_pandas()
dependent_var_df.head()

,category
us_eastern_timestamp,
2024-01-02 04:00:00,C
2024-01-02 04:01:00,C
2024-01-02 04:02:00,C
2024-01-02 04:03:00,C
2024-01-02 04:04:00,C


In [23]:
one_min_df.shape[0]==dependent_var_df.shape[0]

True

### final dataset

In [24]:
one_min_df['category'] = dependent_var_df.category
one_min_df = one_min_df.drop(columns=['us_eastern_timestamp', 'symbol'])
one_min_df

,open,high,low,close,volume,time_delta,open_ema_2m,open_sma_2m,high_ema_2m,high_sma_2m,...,cumulative_local_min_15,time_since_prev_max_15,time_since_prev_min_15,local_max_30,local_min_30,cumulative_local_max_30,cumulative_local_min_30,time_since_prev_max_30,time_since_prev_min_30,category
0,476.25,476.36,476.00,476.31,20460.0,NaN,476.250000,NaN,476.360000,NaN,...,0,0,0,False,False,0,0,0,0,NaN
1,476.34,476.34,476.29,476.29,6369.0,NaN,476.310000,476.295,476.346667,476.350,...,0,1,1,False,False,0,0,1,1,NaN
2,476.29,476.29,476.28,476.28,6152.0,NaN,476.296667,476.315,476.308889,476.315,...,0,2,2,False,False,0,0,2,2,NaN
3,476.27,476.27,476.27,476.27,369.0,NaN,476.278889,476.280,476.282963,476.280,...,0,3,3,False,False,0,0,3,3,NaN
4,476.27,476.27,476.27,476.27,369.0,NaN,476.272963,476.270,476.274321,476.270,...,0,4,4,False,False,0,0,4,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214789,591.53,591.53,591.50,591.52,1052.0,NaN,591.542039,591.530,591.542121,591.530,...,34643,6,0,False,False,27193,23140,6,16,NaN
214790,591.52,591.54,591.48,591.48,3823.0,NaN,591.527346,591.525,591.540707,591.535,...,34644,7,0,False,False,27193,23140,7,17,NaN
214791,591.46,591.46,591.41,591.42,3656.0,NaN,591.482449,591.490,591.486902,591.500,...,34645,8,0,False,False,27193,23140,8,18,NaN
214792,591.40,591.40,591.35,591.39,1354.0,NaN,591.427483,591.430,591.428967,591.430,...,34646,9,0,False,False,27193,23140,9,19,NaN


In [25]:
', '.join(one_min_df.columns)

'open, high, low, close, volume, time_delta, open_ema_2m, open_sma_2m, high_ema_2m, high_sma_2m, low_ema_2m, low_sma_2m, close_ema_2m, close_sma_2m, volume_ema_2m, volume_sma_2m, open_ema_3m, open_sma_3m, high_ema_3m, high_sma_3m, low_ema_3m, low_sma_3m, close_ema_3m, close_sma_3m, volume_ema_3m, volume_sma_3m, open_ema_5m, open_sma_5m, high_ema_5m, high_sma_5m, low_ema_5m, low_sma_5m, close_ema_5m, close_sma_5m, volume_ema_5m, volume_sma_5m, open_ema_8m, open_sma_8m, high_ema_8m, high_sma_8m, low_ema_8m, low_sma_8m, close_ema_8m, close_sma_8m, volume_ema_8m, volume_sma_8m, open_ema_12m, open_sma_12m, high_ema_12m, high_sma_12m, low_ema_12m, low_sma_12m, close_ema_12m, close_sma_12m, volume_ema_12m, volume_sma_12m, open_ema_10m, open_sma_10m, high_ema_10m, high_sma_10m, low_ema_10m, low_sma_10m, close_ema_10m, close_sma_10m, volume_ema_10m, volume_sma_10m, open_ema_20m, open_sma_20m, high_ema_20m, high_sma_20m, low_ema_20m, low_sma_20m, close_ema_20m, close_sma_20m, volume_ema_20m, vol

In [26]:
one_min_df.to_parquet('s3://sisyphus-general-bucket/AthenaInsights/latest_data/model/data/stock_bars_1min.parquet' )

### setting up training

In [28]:
import sagemaker
import boto3
import pandas as pd
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.inputs import TrainingInput

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
# Initialize variables
bucket_name = "sisyphus-general-bucket"
train_features_key = 'AthenaInsights/latest_data/model/data/stock_bars_1min.parquet'
train_labels_key = "AthenaInsights/data/model/train/train_labels.parquet"
s3_output_location = f"s3://{bucket_name}/AthenaInsights/data/model-artifacts"


# Define the S3 paths for the training features and labels
train_features_path = 's3://sisyphus-general-bucket/AthenaInsights/data/model/train/train_features.parquet'
train_labels_path = 's3://sisyphus-general-bucket/AthenaInsights/data/model/train/train_labels.parquet'
